# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:

city_reader = pd.read_csv("city.csv")
temp_city_df = pd.DataFrame(city_reader)
city_df = temp_city_df[["City", "Temp(F)", "Cloudiness", "Humidity", "Wind Speed (mph)", "Latitude", "Longitude","Country"]]
clean_city_df = city_df[city_df["Humidity"] < 100]
clean_city_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
from bleach import clean


gmaps.configure(api_key= g_key)
fig = gmaps.figure()
locations = clean_city_df[["Latitude","Longitude"]]

weights = clean_city_df["Humidity"]


figure = gmaps.figure()
figure.add_layer(gmaps.heatmap_layer(locations, weights = weights, max_intensity = weights.max(), point_radius = 10))

figure


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
hotel_df = clean_city_df[clean_city_df["Cloudiness"] == 0][clean_city_df["Wind Speed (mph)"] < 10][clean_city_df["Temp(F)"] > 60][clean_city_df["Temp(F)"] < 75]
hotel_df = hotel_df.reset_index(drop=True)
hotel_df = hotel_df.drop(index = 24)
hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:

hotel_locations = hotel_df[["Latitude","Longitude"]]
latitudes = hotel_locations["Latitude"].tolist()
longitudes = hotel_locations["Longitude"].tolist()
hotel_latitude = []
hotel_longitude = []
hotel_list = hotel_df["City"].tolist()
hotel_name = []
count = 0
params = {
            "location": f"{latitudes[count]},{longitudes[count]}",
            "keyword": "hotel",
            "type": "hotel",
            "radius": 5000,
            "key": g_key
        }

for name in hotel_list:
    try:
        params = {
            "location": f"{latitudes[count]},{longitudes[count]}",
            "keyword": "hotel",
            "type": "hotel",
            "radius": 5000,
            "key": g_key
        }
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        count += 1
        response = requests.get(base_url, params=params)
        hotel_data = response.json()
        hotel_name.append(hotel_data["results"][0]["name"])
        hotel_latitude.append(hotel_data["results"][0]["geometry"]["location"]["lat"])
        hotel_longitude.append(hotel_data["results"][0]["geometry"]["location"]["lng"])
    except:
        hotel_df = hotel_df.drop(index = count)


In [ ]:
hotel_df["Hotel Name"] = hotel_name

hotel_df["Latitude"] = hotel_latitude
hotel_df["Longitude"] = hotel_longitude
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_filtered_locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_filtered_locations, info_box_content=hotel_info)
figure.add_layer(marker_layer)

# Display figure
figure